In [1]:
from transformers import AutoTokenizer, AutoModelForTokenClassification, pipeline
import spacy
import sys
import gpt3
import cross_coref
import entity_linkage
from pathlib import Path
import csv

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\dgg32\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
error loading _jsonnet (this is expected on Windows), treating C:\Users\dgg32\AppData\Local\Temp\tmpaasw2t50\config.json as plain json
Some weights of the model checkpoint at microsoft/infoxlm-base were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.bias', 'lm_head.dense.weight', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassifica

In [2]:
tokenizer = AutoTokenizer.from_pretrained("pucpr/clinicalnerpt-chemical")

model = AutoModelForTokenClassification.from_pretrained("pucpr/clinicalnerpt-chemical")

model_infer = pipeline('ner',model=model,tokenizer=tokenizer)

nlp = spacy.load("en_core_web_sm")

In [3]:
verb_file = "cazy_keywords.txt"
verbs = open(verb_file).read().split("\n")
verbs

['gh',
 'pl',
 'ce',
 'glycoside hydrolase',
 'polysaccharide lyase',
 'carbohydrate esterase']

In [4]:
cache = {}
local_cache_file = "./cache.tsv"
with open(local_cache_file, newline='') as csvfile:
    name_mapping = csv.reader(csvfile, delimiter='\t')
    for row in name_mapping:
        if len(row) == 2:
            cache[row[0]] = row[1]

In [9]:
#input_file = "./cazy/example_2.txt"

content = f"doi\tprimary\tcazy\taction\tsubstrate\n"

pathlist = Path("./cazy/").glob('*.txt')
for input_file in pathlist:
    #print (input_file)
    with open(input_file, 'r', newline="", encoding='utf-8') as infile:

        for line in infile:
            fields = line.strip().split("|")
            doi = fields[0]
            is_primary = fields[1]
            raw_text = fields[2]

            sentences = (cross_coref.resolve_pronoun(raw_text))

            doc = nlp(sentences)
            assert doc.has_annotation("SENT_START")
            for sent in doc.sents:
                has_verb = False
                for v in verbs:
                    if v in sent.text.lower():
                        has_verb = True
                #print (has_verb)

                if has_verb == True:
                
                    entities = model_infer(sent.text)
                    #print (entities)

                    ### need to recognize some concepts that fall through the cracks of huggingface
                    found_chemical_in_cache = False
                    for key in cache:
                        if key in sent.text:
                            found_chemical_in_cache = True

                    if len(entities) > 0 or found_chemical_in_cache == True:
                        #print (f'"{sent.text.strip()}"')
                        res = gpt3.extract_relation("gpt3_training_cazy.txt", sent.text.strip() + "\n").split("\n")
                        
                        #print(res)

                        for r in res:
                            results = r.split("|")
                            temp_content = ""
                            if len(results) == 3:
                                #print ("hello")
                                #print (f"{doi}|{is_primary}|{results[0]}|{results[1]}|{results[2]}")
                                #print ("results[1]", results[1])
                                disam = entity_linkage.name_disambiguation(results[2])
                                if disam != "":
                                    results[2] = disam

                                temp_content = f'{doi}\t{is_primary}\t' + "\t".join(results)
                            content += temp_content + "\n"

In [10]:
activity_output = open("cazy_literature_extract.tsv", 'w+')
activity_output.write(content)
activity_output.close()